In [6]:
import pandas as pd
import glob
import os

# --- Configurações ---
# O diretório base onde as pastas 5, 10, 15, 20 estão
BASE_PATH =  os.path.join('..','results','imputation','imputed_sample_data')
# As pastas de taxas de missing data que você quer processar
MISSING_RATES = ['5', '10', '15', '20']
# As métricas que você quer na tabela (RMSE e MAE)
METRICS_TO_CALCULATE = ['rmse', 'mae']
# O nome do arquivo de saída
OUTPUT_FILE = 'tabela_latex_resultados.txt'
#-----------------------

def process_imputation_results(base_path, rates, metrics):
    """
    Lê todos os arquivos CSV, calcula as médias e desvios padrão,
    e formata a tabela LaTeX.
    """
    all_data = []
    
    print(f"Iniciando processamento em: {base_path}")

    # 1. Ler todos os arquivos CSV
    for rate in rates:
        dir_path = os.path.join(base_path, rate)
        if not os.path.isdir(dir_path):
            print(f"Aviso: Diretório não encontrado, pulando: {dir_path}")
            continue
            
        # Pega todos os arquivos CSV no diretório
        file_list = glob.glob(os.path.join(dir_path, '*.csv'))
        
        if not file_list:
            print(f"Aviso: Nenhum arquivo .csv encontrado em {dir_path}")
            continue

        print(f"Lendo {len(file_list)} arquivos de {dir_path}...")
        
        for file_path in file_list:
            try:
                df = pd.read_csv(file_path)
                # Adiciona a taxa de missing data como uma coluna
                df['missing_rate'] = int(rate)
                all_data.append(df)
            except Exception as e:
                print(f"Erro ao ler {file_path}: {e}")

    if not all_data:
        print("Erro: Nenhum dado foi lido. Verifique o BASE_PATH.")
        return

    # 2. Combinar tudo em um único DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # 3. Calcular as estatísticas (média e desvio padrão)
    # Agrupando por técnica e taxa de missing data
    stats_df = combined_df.groupby(['imputation_technique', 'missing_rate'])[metrics].agg(['mean', 'std'])
    
    # Reorganizar o DataFrame para facilitar a formatação
    # Isso transforma as 'missing_rate' em colunas
    table_data = stats_df.unstack(level='missing_rate')

    # 4. Gerar a string da tabela LaTeX
    latex_string = generate_latex_table(table_data, rates, metrics)
    
    # 5. Salvar o resultado em um arquivo .txt
    try:
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            f.write(latex_string)
        print(f"\nSucesso! Tabela salva em: {OUTPUT_FILE}")
    except Exception as e:
        print(f"\nErro ao salvar o arquivo: {e}")

def generate_latex_table(table_data, rates, metrics):
    """
    Formata os dados estatísticos em uma string de tabela LaTeX.
    """
    # Lista de técnicas (índice do dataframe)
    techniques = table_data.index
    
    # Inicia a string da tabela
    output_lines = []
    
    # --- Cabeçalho ---
    # Linha 1 (Taxas de Missing Data)
    header_line1 = r"\multirow{2}{*}{\textbf{Imputation Technique}} & "
    cols = []
    for rate in rates:
        cols.append(fr"\multicolumn{{{len(metrics)}}}{{\c}}{{\textbf{{{rate}\%}}}}")
    header_line1 += " & ".join(cols) + r" \\"
    output_lines.append(r"\hline \hline")
    output_lines.append(header_line1)
    
    # Linha 2 (Métricas: RMSE e MAE)
    header_line2 = " & "
    metric_cols = []
    for _ in rates:
        for metric in metrics:
            metric_cols.append(fr"\textbf{{{metric.upper()}}}")
    header_line2 += " & ".join(metric_cols) + r" \\"
    output_lines.append(r"\cline{2-" + str(len(metric_cols) + 1) + "}")
    output_lines.append(header_line2)
    output_lines.append(r"\hline")
    
    # --- Linhas de Dados ---
    for tech in techniques:
        line_parts = [f"{tech.ljust(8)}"] # Nome da técnica com espaçamento
        
        # Acessa os dados da técnica
        tech_data = table_data.loc[tech]
        
        for rate in rates:
            for metric in metrics:
                try:
                    # Tenta pegar média e desvio padrão
                    mean_val = tech_data[(metric, 'mean', int(rate))]
                    std_val = tech_data[(metric, 'std', int(rate))]
                    
                    # Formata como "média ± desvio"
                    cell_text = fr"{mean_val:.2f} $\pm$ {std_val:.2f}"
                    line_parts.append(cell_text)
                    
                except KeyError:
                    # Caso não exista dado para essa combinação
                    print(f"Aviso: Dados não encontrados para {tech}, {rate}%, {metric}")
                    line_parts.append(" - ")
        
        # Junta tudo com o separador de coluna do LaTeX
        output_lines.append(" & ".join(line_parts) + r" \\")

    output_lines.append(r"\hline")
    
    print("\n--- Pré-visualização da Tabela ---")
    print("\n".join(output_lines))
    
    return "\n".join(output_lines)

# --- Ponto de entrada do Script ---
if __name__ == "__main__":
    # Verifica se o pandas está instalado
    try:
        import pandas
    except ImportError:
        print("Erro: A biblioteca 'pandas' não está instalada.")
        print("Por favor, instale com: pip install pandas")
        exit()
        
    process_imputation_results(BASE_PATH, MISSING_RATES, METRICS_TO_CALCULATE)

Iniciando processamento em: ..\results\imputation\imputed_sample_data
Lendo 10 arquivos de ..\results\imputation\imputed_sample_data\5...
Lendo 10 arquivos de ..\results\imputation\imputed_sample_data\10...
Lendo 10 arquivos de ..\results\imputation\imputed_sample_data\15...
Lendo 10 arquivos de ..\results\imputation\imputed_sample_data\20...

--- Pré-visualização da Tabela ---
\hline \hline
\multirow{2}{*}{\textbf{Imputation Technique}} & \multicolumn{2}{\c}{\textbf{5\%}} & \multicolumn{2}{\c}{\textbf{10\%}} & \multicolumn{2}{\c}{\textbf{15\%}} & \multicolumn{2}{\c}{\textbf{20\%}} \\
\cline{2-9}
 & \textbf{RMSE} & \textbf{MAE} & \textbf{RMSE} & \textbf{MAE} & \textbf{RMSE} & \textbf{MAE} & \textbf{RMSE} & \textbf{MAE} \\
\hline
ARIMA    & 6.26 $\pm$ 4.52 & 1.02 $\pm$ 0.77 & 17.84 $\pm$ 11.96 & 4.26 $\pm$ 3.02 & 22.36 $\pm$ 11.54 & 6.55 $\pm$ 3.35 & 25.74 $\pm$ 12.63 & 9.15 $\pm$ 4.70 \\
HoltWinters & 5.47 $\pm$ 4.86 & 0.93 $\pm$ 0.84 & 16.62 $\pm$ 10.43 & 3.86 $\pm$ 2.47 & 19.71 $\pm$

In [14]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configurações ---
# O diretório base onde as pastas 5, 10, 15, 20 estão
BASE_PATH = os.path.join('..', 'results', 'imputation', 'imputed_sample_data')
# As pastas de taxas de missing data que você quer processar
MISSING_RATES = ['5', '10', '15', '20']
# As métricas que você quer nos gráficos
METRICS_TO_PLOT = ['fam', 'dtw_magnitude', 'dtw_shape']
# O diretório onde os gráficos serão salvos
OUTPUT_DIR = 'visualizacoes_resultados'
#-----------------------

def load_and_prepare_data(base_path, rates, metrics):
    """
    Lê todos os arquivos CSV de todos os datasets e taxas,
    combina-os em um DataFrame e o "derrete" (melts) para
    o formato longo, ideal para o Seaborn.
    """
    all_data = []
    
    print(f"Iniciando processamento em: {base_path}")

    # 1. Ler todos os arquivos CSV
    for rate in rates:
        dir_path = os.path.join(base_path, rate)
        if not os.path.isdir(dir_path):
            print(f"Aviso: Diretório não encontrado, pulando: {dir_path}")
            continue
            
        file_list = glob.glob(os.path.join(dir_path, '*.csv'))
        
        if not file_list:
            print(f"Aviso: Nenhum arquivo .csv encontrado em {dir_path}")
            continue

        print(f"Lendo {len(file_list)} arquivos de {dir_path}...")
        
        for file_path in file_list:
            try:
                # Extrai o nome do dataset do nome do arquivo
                # Pega tudo antes do primeiro "_"
                basename = os.path.basename(file_path)
                dataset_name = basename.split('_')[0]
                
                df = pd.read_csv(file_path)
                
                # Adiciona as colunas de taxa e dataset
                df['missing_rate'] = int(rate)
                df['dataset'] = dataset_name
                
                # Verifica se as métricas existem
                if all(metric in df.columns for metric in metrics):
                    all_data.append(df)
                else:
                    print(f"Aviso: Pulando {file_path}, nem todas as métricas {metrics} foram encontradas.")
                    
            except Exception as e:
                print(f"Erro ao ler {file_path}: {e}")

    if not all_data:
        print("Erro: Nenhum dado foi lido. Verifique o BASE_PATH.")
        return None

    # 2. Combinar tudo em um único DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # 3. "Derreter" (Melt) o DataFrame
    # Isso transforma as colunas de métrica (fam, dtw_mag, dtw_shape)
    # em linhas, facilitando a plotagem com 'hue' ou 'row'.
    
    id_vars = ['dataset', 'missing_rate', 'imputation_technique']
    # Garante que apenas as colunas necessárias estão presentes
    columns_to_melt = id_vars + metrics
    
    # Filtra o DataFrame para evitar colunas desnecessárias no melt
    try:
        meltable_df = combined_df[columns_to_melt]
    except KeyError as e:
        print(f"Erro: Coluna faltando no DataFrame. Verifique se os CSVs contêm as colunas: {e}")
        print("Colunas encontradas:", combined_df.columns)
        return None
        
    long_df = meltable_df.melt(
        id_vars=id_vars,
        value_vars=metrics,
        var_name='metric',
        value_name='value'
    )
    
    print(f"\nDados carregados e preparados com {len(long_df)} linhas.")
    return long_df

def plot_line_charts(df, output_dir):
    """
    Solução 1: Gráficos de Linha (Recomendado)
    Gera uma grade de gráficos de linha, separados por dataset e métrica.
    """
    print("Gerando gráficos de linha (Solução 1)...")
    try:
        g = sns.relplot(
            data=df,
            x="missing_rate",
            y="value",
            hue="imputation_technique",
            style="imputation_technique", # Estilos de linha diferentes
            markers=True,
            col="dataset",
            row="metric",
            kind="line",
            legend="full",
            facet_kws={'sharey': False}, # Deixa o eixo Y livre
            height=4,
            aspect=1.2
        )
        
        # Ajusta os títulos
        g.set_titles("Dataset: {col_name} | Métrica: {row_name}")
        g.set_axis_labels("Taxa de Dados Faltantes (%)", "Valor da Métrica")
        
        output_path = os.path.join(output_dir, "graficos_de_linha_por_dataset.png")
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Gráficos de linha salvos em: {output_path}")

    except Exception as e:
        print(f"Erro ao gerar gráficos de linha: {e}")

def plot_bar_charts(df, output_dir):
    """
    Solução 2: Gráficos de Barras Agrupados
    Gera um arquivo de imagem separado PARA CADA DATASET.
    """
    print("Gerando gráficos de barras (Solução 2)...")
    datasets = df['dataset'].unique()
    
    for dataset in datasets:
        print(f"  - Processando barras para: {dataset}")
        try:
            dataset_df = df[df['dataset'] == dataset]
            
            g = sns.catplot(
                data=dataset_df,
                x="imputation_technique",
                y="value",
                hue="missing_rate",
                row="metric",
                kind="bar",
                legend="full",
                facet_kws={'sharey': False},
                height=4,
                aspect=2.5
            )
            
            g.set_titles("Dataset: " + dataset + " | Métrica: {row_name}")
            g.set_axis_labels("Técnica de Imputação", "Valor da Métrica")
            g.set_xticklabels(rotation=45, horizontalalignment='right')
            
            output_path = os.path.join(output_dir, f"grafico_barras_{dataset}.png")
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            plt.close()
            
        except Exception as e:
            print(f"Erro ao gerar gráfico de barras para {dataset}: {e}")
    
    print(f"Gráficos de barras salvos em: {output_dir}")

def plot_heatmaps(df, output_dir):
    """
    Solução 3: Mapas de Calor (Heatmaps)
    Gera um arquivo de imagem separado PARA CADA DATASET e CADA MÉTRICA.
    """
    print("Gerando heatmaps (Solução 3 - Por Dataset)...")
    datasets = df['dataset'].unique()
    metrics = df['metric'].unique()
    
    for dataset in datasets:
        for metric in metrics:
            print(f"  - Processando heatmap para: {dataset} / {metric}")
            try:
                # Filtra os dados
                subset = df[(df['dataset'] == dataset) & (df['metric'] == metric)]
                
                # Pivota os dados para o formato do heatmap
                pivot_df = subset.pivot(
                    index="imputation_technique",
                    columns="missing_rate",
                    values="value"
                )
                
                # Define o esquema de cores (invertido para FAM)
                cmap = "viridis_r" if metric == 'fam' else "viridis"
                
                # Cria a figura
                plt.figure(figsize=(10, 8))
                sns.heatmap(
                    pivot_df,
                    annot=True,     # Escreve o valor na célula
                    fmt=".2f",      # Formato com 2 casas decimais
                    linewidths=.5,
                    cmap=cmap
                )
                
                plt.title(f"Heatmap - Métrica: {metric.upper()} | Dataset: {dataset}")
                plt.xlabel("Taxa de Dados Faltantes (%)")
                plt.ylabel("Técnica de Imputação")
                
                output_path = os.path.join(output_dir, f"heatmap_{dataset}_{metric}.png")
                plt.savefig(output_path, dpi=300, bbox_inches='tight')
                plt.close()
                
            except Exception as e:
                print(f"Erro ao gerar heatmap para {dataset} / {metric}: {e}")

    print(f"Heatmaps por dataset salvos em: {output_dir}")

def plot_combined_heatmaps(df, output_dir):
    """
    Solução 4: Mapas de Calor Combinados
    Gera um heatmap por métrica, combinando todos os datasets.
    """
    print("Gerando heatmaps (Solução 4 - Combinado)...")
    metrics = df['metric'].unique()
    
    for metric in metrics:
        print(f"  - Processando heatmap combinado para: {metric}")
        try:
            # Filtra os dados
            subset = df[df['metric'] == metric].copy()
            
            # Cria uma nova coluna combinando dataset e técnica
            subset['dataset_technique'] = subset['dataset'] + ' - ' + subset['imputation_technique']
            
            # Pivota os dados
            pivot_df = subset.pivot(
                index="dataset_technique",
                columns="missing_rate",
                values="value"
            )
            
            # Ordena o índice para agrupar datasets
            pivot_df = pivot_df.sort_index()
            
            # Define o esquema de cores (invertido para FAM)
            cmap = "viridis_r" if metric == 'fam' else "viridis"
            
            # Calcula a altura dinâmica do gráfico
            num_rows = len(pivot_df.index)
            fig_height = max(10, num_rows * 0.4) # 0.4 polegadas por linha
            
            # Cria a figura
            plt.figure(figsize=(12, fig_height))
            sns.heatmap(
                pivot_df,
                annot=True,
                fmt=".2f",
                linewidths=.5,
                cmap=cmap
            )
            
            plt.title(f"Heatmap Combinado - Métrica: {metric.upper()}")
            plt.xlabel("Taxa de Dados Faltantes (%)")
            plt.ylabel("Dataset - Técnica de Imputação")
            
            output_path = os.path.join(output_dir, f"heatmap_combinado_{metric}.png")
            plt.savefig(output_path, dpi=300, bbox_inches='tight')
            plt.close()
            
        except Exception as e:
            print(f"Erro ao gerar heatmap combinado para {metric}: {e}")
            
    print(f"Heatmaps combinados salvos em: {output_dir}")

def process_and_visualize_results(base_path, rates, metrics, output_dir):
    """
    Função principal para orquestrar o carregamento e a geração
    de todos os gráficos.
    """
    # 1. Carregar e preparar os dados
    long_df = load_and_prepare_data(base_path, rates, metrics)
    
    if long_df is None:
        print("Processamento interrompido pois nenhum dado foi carregado.")
        return

    # 2. Criar o diretório de saída se não existir
    if not os.path.exists(output_dir):
        print(f"Criando diretório de saída: {output_dir}")
        os.makedirs(output_dir)

    # 3. Gerar os gráficos
    plot_line_charts(long_df.copy(), output_dir)
    plot_bar_charts(long_df.copy(), output_dir)
    plot_heatmaps(long_df.copy(), output_dir)
    plot_combined_heatmaps(long_df.copy(), output_dir) # <-- Nova função adicionada
    
    print("\nProcessamento de visualização concluído!")

# --- Ponto de entrada do Script ---
if __name__ == "__main__":
    # Verifica se as bibliotecas necessárias estão instaladas
    try:
        import pandas
        import matplotlib
        import seaborn
    except ImportError as e:
        print(f"Erro: Biblioteca necessária não instalada: {e.name}")
        print("Por favor, instale as bibliotecas com:")
        print("pip install pandas matplotlib seaborn")
        exit()
        
    process_and_visualize_results(BASE_PATH, MISSING_RATES, METRICS_TO_PLOT, OUTPUT_DIR)



Iniciando processamento em: ..\results\imputation\imputed_sample_data
Lendo 10 arquivos de ..\results\imputation\imputed_sample_data\5...
Lendo 10 arquivos de ..\results\imputation\imputed_sample_data\10...
Lendo 10 arquivos de ..\results\imputation\imputed_sample_data\15...
Lendo 10 arquivos de ..\results\imputation\imputed_sample_data\20...

Dados carregados e preparados com 1200 linhas.
Criando diretório de saída: visualizacoes_resultados
Gerando gráficos de linha (Solução 1)...
Gráficos de linha salvos em: visualizacoes_resultados\graficos_de_linha_por_dataset.png
Gerando gráficos de barras (Solução 2)...
  - Processando barras para: esmond data perfsonar-ankara.ulakbim.gov.tr to psmp-gn-bw-lis-pt.geant.org 10-24-2023
Erro ao gerar gráfico de barras para esmond data perfsonar-ankara.ulakbim.gov.tr to psmp-gn-bw-lis-pt.geant.org 10-24-2023: seaborn.axisgrid.FacetGrid() got multiple values for keyword argument 'sharey'
  - Processando barras para: esmond data perfsonar-ankara.ulakbim